# Spotify top 200 charts (2020 - 2021)
### This data sets is goinfg to be used to identify patterns in the genres and key in which popular music is composed

### Todos:
- [x] Read csv
- [x] Format csv into a list with only the required info (position, name, artist, genre, tempo, duration and key)
- []  


In [59]:
import numpy as np
import matplotlib.pyplot as plt
import csv

def read_csv(csv_route):
  with open(csv_route, encoding = "cp850") as file:
    reader = csv.reader(file, delimiter=",")
    data_indices = [1, 4, 6, 9, 19, 20, 22]
    float_indices = [0, 4, 5]
    header = next(reader)
    header = [header[i] for i in data_indices]
    data = []
    for row in reader:
      song = [row[i] for i in data_indices]
      data.append(song)
    for idx, song in enumerate(data):
      for i, val in enumerate(song):
        if val == " ":
          del data[idx]
        else: 
          if i in float_indices:
            song[i] = float(song[i])
    return data, header

def filter_data(data, upper_limit, lower_limit):
  return list(filter(lambda x: float(x[0]) <= upper_limit and float(x[0]) >= lower_limit, data))

data, header = read_csv('./spotify_dataset.csv')
top_five = filter_data(data, 200, 199)

print(top_five)

fig, ax = plt.subplots()

[[199.0, 'Verdansk', 'Dave', "['uk hip hop']", 117.078, 182285.0, 'D'], [200.0, 'Vida Louca', 'Mc Poze do Rodo, Neo Beats, Mainstreet', "['funk carioca', 'funk ostentacao']", 92.464, 155676.0, 'F#/Gb'], [199.0, '10 Years', 'Da├░i Freyr', "['icelandic pop', 'sunnlensk tonlist']", 123.038, 166000.0, 'D'], [199.0, 'Paradigmas', 'Jorge & Mateus', "['sertanejo', 'sertanejo universitario']", 157.932, 157355.0, 'C#/Db'], [200.0, 'LOST', 'NF, Hopsin', "['hip hop', 'pop rap', 'rap']", 116.919, 234852.0, 'F'], [199.0, 'Conf├¡a', 'Sech, Daddy Yankee', "['latin', 'panamanian pop', 'reggaeton', 'trap latino']", 167.987, 202308.0, 'C'], [199.0, 'Lover (Remix) [feat. Shawn Mendes]', 'Taylor Swift', "['pop', 'post-teen pop']", 205.272, 221307.0, 'G']]
